In [102]:
import urllib
import urllib.request
import re

In [103]:
# Base url we will use: https://sr.wikisource.org/sr-ec/Српске_епске_народне_песме (without encoding)
page = urllib.request.urlopen('https://sr.wikisource.org/sr-ec/%D0%A1%D1%80%D0%BF%D1%81%D0%BA%D0%B5_%D0%B5%D0%BF%D1%81%D0%BA%D0%B5_%D0%BD%D0%B0%D1%80%D0%BE%D0%B4%D0%BD%D0%B5_%D0%BF%D0%B5%D1%81%D0%BC%D0%B5')

In [104]:
# Extract all the links
regex = re.compile(r'<a href="[^"]*?" title="[^"]*?">([^>]*.?)</a>')
content = page.read().decode()

In [105]:
# Find all the links
results = regex.findall(content)
# Take only links for epic songs (this part is hardcoded for the sake of speed)
results = results[1:-15]
results = list(filter(lambda x: x != 'уреди', results))

In [106]:
# We wish to map this map with shape:
# song_title: song_content
songs = {}

In [107]:
def encode_url(base_url, song_name):
    """ Encodes the url and returns the encoded url. """
    return base_url + urllib.parse.quote_plus(song_name.replace(' ', '_').encode('utf-8'), '/:')

In [108]:
# Test the function
base_url = 'https://sr.wikisource.org/wiki/'
print("Encoded url: {}".format(encode_url(base_url, results[0])))

Encoded url: https://sr.wikisource.org/wiki/%D0%91%D0%BE%D0%B3_%D0%BD%D0%B8%D0%BA%D0%BE%D0%BC_%D0%B4%D1%83%D0%B6%D0%B0%D0%BD_%D0%BD%D0%B5_%D0%BE%D1%81%D1%82%D0%B0%D1%98%D0%B5


In [109]:
from bs4 import BeautifulSoup

# TODO: remove html comments
def clean_of_html(content):
    # Remove random numbers for line counting
    content = re.sub('\d+ ', '', content)
    
    content = re.sub('„ „', '„', content)
    
     # Remove spans (used for some internal formating of lines on website)
    content = re.sub('<span.*?>.*?</span>', '', content)
    
    # Strip all html tags but keep their values in between the tags
    content = BeautifulSoup(content, "lxml").text
    
    # Some random junk we html left us with
    content = content.replace('Напомене[уреди]', '')
    content = content.replace('- - - - - - - - - - - - - - - - - - - - -', '')
    
    # Replace multiple newlines with only one
    content = re.sub('\n+', '\n', content)
    
    
    return content

In [110]:
candidates_to_fix = []
total_songs = len(results)
current = 1

# Iterate over songs, extract their content and save it to map
for song_name in results:
    url = encode_url(base_url, song_name)
    page = urllib.request.urlopen(url)
    content = page.read().decode()
    
    # Regex that extracts the song content
    song_re = re.compile(r'<div class="mw-parser-output">(.*?)</div>', re.DOTALL)
    
    for r in song_re.finditer(content):
        element = clean_of_html(r.group(1))
        songs[song_name] = element
        print("Saved song {0}/{1} {2} with length of {3}".format(current, total_songs, song_name, len(element)))
        
        if (len(element) < 50):
            candidates_to_fix.append((song_name, len(element)))
    
    current += 1                          
            
print('Should fix: ')
for (k, v) in candidates_to_fix: print(k, v)

Saved song 1/118 Бог ником дужан не остаје with length of 3600
Saved song 2/118 Ко крсно име слави, оном и помаже with length of 3086
Saved song 3/118 Опет то, али друкчије with length of 2418
Saved song 4/118 Свети Никола with length of 1145
Saved song 5/118 Ђакон Стефан и два анђела with length of 3492
Saved song 6/118 Заручница Лаза Радановића with length of 4437
Saved song 7/118 Кумовање Грчића Манојла with length of 2030
Saved song 8/118 Дунав се Савом оженио with length of 1150
Saved song 9/118 Бан Милутин и Дука Херцеговац with length of 10726
Saved song 10/118 Бановић Страхиња with length of 23162
Saved song 11/118 Дарови светог Јована Владимира with length of 713
Saved song 12/118 Дијете Јован и ћерка цара Стефана with length of 8905
Saved song 13/118 Душан хоће сестру да узме with length of 3887
Saved song 14/118 Женидба Душанова with length of 19491
Saved song 15/118 Женидба кнеза Лазара with length of 4681
Saved song 16/118 Женидба краља Вукашина with length of 1
Saved song

In [111]:
# Handle few songs with different formatting
for (song_name, _) in candidates_to_fix:
    url = encode_url(base_url, song_name)
    page = urllib.request.urlopen(url)
    content = page.read().decode()
    prob_song_re = re.compile('<div class="poem">(.*?)</div>', re.DOTALL)
    for r in prob_song_re.finditer(content):
        song_content = clean_of_html(r.group(1))
        songs[song_name] = song_content
        print(song_content)

Књигу пише жура Вукашине
у бијелу Скадру на Бојани,
те је шаље на Херцеговину
бијеломе граду Пирлитору,
Пирлитору према Дурмитору,
Видосави, љуби Момчиловој;
тајно пише, а тајно јој шаље,
у књизи јој овако бесједи:
„Видосава, Момчилова љубо,
шта ћеш у том леду и снијегу?
Кад погледаш с града изнад себе,
ништа немаш лијепо виђети,
већ бијело брдо Дурмитора,
окићено ледом и снијегом
усред љета као усред зиме;
кад погледаш стрмо испод града,
мутна тече Тара валовита,
она ваља дрвље и камење,
на њој нема брода ни ћуприје,
а око ње борје и мраморје;
већ ти отруј војводу Момчила,
ил' га отруј, или ми га издај,
ходи к мени у приморје равно,
бијеломе Скадру на Бојану,
узећу те за вјерну љубовцу,
па ћеш бити госпођа краљица,
прести свилу на златно вретено,
свилу прести, на свили сједити,
а носити диву и кадиву
и још оно све жежено злато.
А какав је Скадар на Бојани!
Кад погледаш брду изнад града,
све порасле смокве и маслине
и још они грозни виногради;
кад погледаш стрмо испод града,
ал' узрасл

In [115]:
current = 1
total_songs = len(songs)
with open('epic_songs.txt', 'w') as f:
    for (k, v) in songs.items():
        f.write(v + '\n')
        print('Wrote {0}/{1} {2}'.format(current, total_songs, k))
        current += 1

Wrote 1/118 Бог ником дужан не остаје
Wrote 2/118 Ко крсно име слави, оном и помаже
Wrote 3/118 Опет то, али друкчије
Wrote 4/118 Свети Никола
Wrote 5/118 Ђакон Стефан и два анђела
Wrote 6/118 Заручница Лаза Радановића
Wrote 7/118 Кумовање Грчића Манојла
Wrote 8/118 Дунав се Савом оженио
Wrote 9/118 Бан Милутин и Дука Херцеговац
Wrote 10/118 Бановић Страхиња
Wrote 11/118 Дарови светог Јована Владимира
Wrote 12/118 Дијете Јован и ћерка цара Стефана
Wrote 13/118 Душан хоће сестру да узме
Wrote 14/118 Женидба Душанова
Wrote 15/118 Женидба кнеза Лазара
Wrote 16/118 Женидба краља Вукашина
Wrote 17/118 Зидање Раванице
Wrote 18/118 Зидање Раванице, опет
Wrote 19/118 Зидање Скадра
Wrote 20/118 Како се крсно име служи
Wrote 21/118 Краљ Владимир и Свети Наум
Wrote 22/118 Милан-бег и Драгутин-бег
Wrote 23/118 Милош у Латинима
Wrote 24/118 Наход Момир
Wrote 25/118 Песма о краљу Владимиру
Wrote 26/118 Пут краља Владимира
Wrote 27/118 Свети Саво
Wrote 28/118 Смрт Душанова
Wrote 29/118 Смрт цара Урош

In [114]:
len({'x':5})

1